In [1]:
import pandas as pd

# the file is first explored, showing a completely messy DataFrame
data = pd.read_excel('2018-2.xlsx')
data.head()

,DUMMY DATA,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,This cell contains more data,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Here is a table that is not important:,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,col1,col2,col3,col4,NaN,NaN
4,NaN,name,value,-51907,8.53,NaN,NaN


In [2]:
# Taking a quick peek into the Excel file, it is noted that the first table with the relevant information 
# starts from the 17nth row
# thus, it is convenient to skip importing the first 16 rows
# row number 17 is taken as the header for the DataFrame
data = pd.read_excel('2018-2.xlsx', skiprows=16)
data.head()

,CODE,DATE,NAME,AMOUNT,TAX,INTEREST RATE,COST
0,000000,19/02/2012,Et harum quidem rerum,"23,068",,,"81,530.00"
1,A2519D,'29/01/2013,Nam libero tempore,"67,147.00","0,00","00,00","27,647.00"
2,R81963,30/12/2014,cum soluta nobis,"81,027.00-",,,"27,803.00-"
3,,NaN,NaN,NaN,NaN,NaN,NaN
4,"""Lorem ipsum dolor sit amet, consectetur adipi...",NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Rows with null values will be skipped
# axis=0 states that the rows are the ones to be dropped
# spoiler alert: the DataFrame header is found into de dataset, as part of the second table
data.dropna(axis=0, how='any', inplace=True)
data.head()

,CODE,DATE,NAME,AMOUNT,TAX,INTEREST RATE,COST
0,000000,19/02/2012,Et harum quidem rerum,"23,068",,,"81,530.00"
1,A2519D,'29/01/2013,Nam libero tempore,"67,147.00","0,00","00,00","27,647.00"
2,R81963,30/12/2014,cum soluta nobis,"81,027.00-",,,"27,803.00-"
27,CODE,DATE,NAME,AMOUNT,TAX,INTEREST RATE,COST
28,000000,12/02/2018,Itaque earum rerum,0.42-,,,0.32-


In [4]:
# do not take into account rows which contain the same value as the corresponding column name, 
# or DataFrame header. Taking just first column as example, but you may iterate over column names
# getting sure none of the column headers are in the dataset
# as a second condition, I have stated to drop rows wich are empty, or contain just a blank space
data = data[(data.columns[0] != data[data.columns[0]]) & (data[data.columns[0]] != ' ')]
data

,CODE,DATE,NAME,AMOUNT,TAX,INTEREST RATE,COST
0,000000,19/02/2012,Et harum quidem rerum,"23,068",,,"81,530.00"
1,A2519D,'29/01/2013,Nam libero tempore,"67,147.00","0,00","00,00","27,647.00"
2,R81963,30/12/2014,cum soluta nobis,"81,027.00-",,,"27,803.00-"
28,000000,12/02/2018,Itaque earum rerum,0.42-,,,0.32-
29,000000,12/03/2017,tenetur a sapiente delectus,0.42,,,0.82
30,I018010,10/04/2016,At vero eos et accusamus,4.02,"1,50","27,49",0.17
31,004057,06/05/2015,Sed ut perspiciatis unde omnis,0.05-,,,0.95-
32,010503,06/06/2014,Nemo enim ipsam voluptatem,0.05,,,0.35
33,K47149,02/07/2013,Ut enim ad minima veniam,0.05,,,0.60
34,Q72148,29/08/2012,nisi ut aliquid ex ea commodi,9.00-,,,9.20-


In [5]:
# both columns AMOUNT and COST have the negative sign at the end. 
a = data['AMOUNT'].str.split('-', expand=True)
a[1] = a[1].str.replace('', '-')
a[1] = a[1].fillna('')
data['AMOUNT'] = a[1] + a[0]

a = data['COST'].str.split('-', expand=True)
a[1] = a[1].str.replace('', '-')
a[1] = a[1].fillna('')
data['COST'] = (a[1] + a[0])
data

,CODE,DATE,NAME,AMOUNT,TAX,INTEREST RATE,COST
0,000000,19/02/2012,Et harum quidem rerum,"23,068",,,"81,530.00"
1,A2519D,'29/01/2013,Nam libero tempore,"67,147.00","0,00","00,00","27,647.00"
2,R81963,30/12/2014,cum soluta nobis,"-81,027.00",,,"-27,803.00"
28,000000,12/02/2018,Itaque earum rerum,-0.42,,,-0.32
29,000000,12/03/2017,tenetur a sapiente delectus,0.42,,,0.82
30,I018010,10/04/2016,At vero eos et accusamus,4.02,"1,50","27,49",0.17
31,004057,06/05/2015,Sed ut perspiciatis unde omnis,-0.05,,,-0.95
32,010503,06/06/2014,Nemo enim ipsam voluptatem,0.05,,,0.35
33,K47149,02/07/2013,Ut enim ad minima veniam,0.05,,,0.60
34,Q72148,29/08/2012,nisi ut aliquid ex ea commodi,-9.00,,,-9.20


In [6]:
# The comma (",") is going to be problematic for the conversion into floating number. 
# It is best to remove it
# Data conversion is done for every number-type column
data['AMOUNT'] = data['AMOUNT'].str.replace(',', '').astype('float')
data['COST'] = data['COST'].str.replace(',', '').astype('float')
data['TAX'] = data['TAX'].str.replace(',', '.').replace(' ', 'nan').astype('float')
data['INTEREST RATE'] = data['INTEREST RATE'].str.replace(',', '.').replace(' ', 'nan').astype('float')

# The dates are also formatted for beter representation and ease of processing
data['DATE'] = data['DATE'].str.replace("'", '')
data['DATE'] = pd.to_datetime(data['DATE'], format="%d/%m/%Y")
data

,CODE,DATE,NAME,AMOUNT,TAX,INTEREST RATE,COST
0,000000,2012-02-19,Et harum quidem rerum,23068.00,NaN,NaN,81530.00
1,A2519D,2013-01-29,Nam libero tempore,67147.00,0.0,0.00,27647.00
2,R81963,2014-12-30,cum soluta nobis,-81027.00,NaN,NaN,-27803.00
28,000000,2018-02-12,Itaque earum rerum,-0.42,NaN,NaN,-0.32
29,000000,2017-03-12,tenetur a sapiente delectus,0.42,NaN,NaN,0.82
30,I018010,2016-04-10,At vero eos et accusamus,4.02,1.5,27.49,0.17
31,004057,2015-05-06,Sed ut perspiciatis unde omnis,-0.05,NaN,NaN,-0.95
32,010503,2014-06-06,Nemo enim ipsam voluptatem,0.05,NaN,NaN,0.35
33,K47149,2013-07-02,Ut enim ad minima veniam,0.05,NaN,NaN,0.60
34,Q72148,2012-08-29,nisi ut aliquid ex ea commodi,-9.00,NaN,NaN,-9.20
